In [2]:
import pandas as pd

In [3]:
fin_data = pd.read_csv('./dataset/finantials.csv')
movie_data = pd.read_csv('./dataset/movies.csv')
opening_data = pd.read_csv('./dataset/opening_gross.csv')

In [5]:
numeric_columns_mask = (movie_data.dtypes==float) | (movie_data.dtypes==int)  #indica qué columnas en el DataFrame movie_data son de tipo float o int
numeric_columns = [column for column in numeric_columns_mask.index if numeric_columns_mask[column]]   #Se está creando una lista llamada numeric_columns que contiene los nombres de las columnas que cumplen con la condición especificada por la máscara numeric_columns_mask. En otras palabras, esta línea extrae los nombres de las columnas que son de tipo float o int según la máscara booleana.
movie_data = movie_data[numeric_columns+['movie_title']]   #Finalmente, se está actualizando el DataFrame movie_data para incluir solo las columnas que se encuentran en la lista numeric_columns más la columna 'movie_title'. Esto es una forma de seleccionar solo las columnas numéricas y la columna 'movie_title' en el DataFrame original.

In [6]:
fin_data = fin_data[['movie_title','production_budget','worldwide_gross']]  # esta línea de código selecciona y mantiene solo las columnas relevantes en el DataFrame fin_data, dejando solo las columnas 'movie_title', 'production_budget' y 'worldwide_gross' para su posterior procesamiento o análisis. 

In [7]:
fin_movie_data = pd.merge(fin_data, movie_data, on='movie_title',how='left')  #Aquí, se está combinando el DataFrame fin_data con el DataFrame movie_data utilizando la columna 'movie_title' como clave de combinación. La opción how='left' indica que se realizará un left join, es decir, se conservarán todas las filas de fin_data y solo se agregarán las columnas de movie_data que coincidan con las 'movie_title' correspondientes. El resultado se almacena en el nuevo DataFrame llamado fin_movie_data.

full_movie_data = pd.merge(opening_data, fin_movie_data, on='movie_title', how='left')

In [8]:
full_movie_data[(full_movie_data.worldwide_gross != 0) & (full_movie_data.worldwide_gross.notnull())].shape

(2304, 11)

In [9]:
full_movie_data = full_movie_data.drop(['movie_title','gross'], axis=1)

In [10]:
full_movie_data.columns

Index(['opening_gross', 'screens', 'production_budget', 'worldwide_gross',
       'title_year', 'aspect_ratio', 'duration', 'budget', 'imdb_score'],
      dtype='object')

Modeling

In [25]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_validate, GridSearchCV, train_test_split
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import numpy as np

In [26]:
X = full_movie_data.drop(['worldwide_gross'], axis=1)
y = full_movie_data['worldwide_gross']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35, random_state=42)

In [27]:
original_params = {
    'n_estimators': 220,
    'alpha': 0.9,
    'ccp_alpha': 0.0,
    'criterion': 'friedman_mse',
    'init': None,
    'learning_rate': 0.1,
    'loss': 'squared_error',
    'max_depth': 3,
    'max_features': None,
    'max_leaf_nodes': None,
    'min_impurity_decrease': 0.0,
    'min_samples_leaf': 1,
    'min_samples_split': 2,
    'min_weight_fraction_leaf': 0.0,
    'n_iter_no_change': None,
    'random_state': None,
    'subsample': 1.0,
    'tol': 0.0001,
    'validation_fraction': 0.1,
    'verbose': 0,
    'warm_start': False
}

estimator = Pipeline([
    ('imputer', SimpleImputer(missing_values=np.nan, strategy='mean')),
    ('scaler', StandardScaler()),
    ('core_model', GradientBoostingRegressor(**original_params))
])

Este código define un diccionario llamado `original_params` que contiene los parámetros y sus valores asociados para configurar un modelo de regresión con Gradient Boosting utilizando la clase `GradientBoostingRegressor` de la biblioteca scikit-learn.

Vamos a desglosar los parámetros y sus significados:

- `n_estimators`: Número de árboles de decisión que se construirán. En este caso, se establece en 220.

- `alpha`: Factor de regularización para la función de pérdida. En este caso, se establece en 0.9.

- `ccp_alpha`: Parámetro de complejidad usado para la poda minimalista de costo-complejidad. En este caso, se establece en 0.0.

- `criterion`: La función para medir la calidad de una división. En este caso, se utiliza 'friedman_mse', que es la desviación media cuadrática con mejoras específicas de Friedman.

- `init`: Un estimador de regresión que se utiliza para inicializar el conjunto. En este caso, se establece en `None`, lo que significa que se usará un árbol débil como estimador inicial.

- `learning_rate`: Tasa de aprendizaje, controla la contribución de cada árbol al conjunto. En este caso, se establece en 0.1.

- `loss`: La función de pérdida a optimizar. En este caso, se utiliza 'squared_error', que corresponde a la regresión de mínimos cuadrados.

- `max_depth`: Profundidad máxima de los árboles de decisión. En este caso, se establece en 3.

- `max_features`: Número máximo de características consideradas para dividir un nodo. En este caso, se establece en `None`, lo que significa que se consideran todas las características.

- `max_leaf_nodes`: Número máximo de nodos hoja. En este caso, se establece en `None`, lo que significa que no hay una restricción en el número de nodos hoja.

- `min_impurity_decrease`: Un nodo se dividirá si esta división induce una disminución de la impureza mayor o igual a este valor. En este caso, se establece en 0.0.

- `min_samples_leaf`: Número mínimo de muestras requeridas para estar en un nodo hoja. En este caso, se establece en 1.

- `min_samples_split`: Número mínimo de muestras requeridas para dividir un nodo interno. En este caso, se establece en 2.

- `min_weight_fraction_leaf`: La fracción mínima ponderada de la suma total de pesos de las muestras necesarias para estar en un nodo hoja. En este caso, se establece en 0.0.

- `n_iter_no_change`: Número máximo de iteraciones sin mejora para detener el ajuste temprano. En este caso, se establece en `None`, lo que significa que no se utiliza el ajuste temprano.

- `random_state`: Controla la aleatoriedad del estimador. En este caso, se establece en `None`, lo que significa que no se establece una semilla aleatoria específica.

- `subsample`: Fracción de muestras utilizadas para ajustar cada árbol. En este caso, se establece en 1.0, lo que significa que se utiliza el conjunto de datos completo.

- `tol`: Tolerancia a la mejora en la función objetivo. En este caso, se establece en 0.0001.

- `validation_fraction`: Fracción de datos de entrenamiento a utilizar para la validación temprana. En este caso, se establece en 0.1.

- `verbose`: Controla la cantidad de información que se imprime. En este caso, se establece en 0 para no imprimir ninguna información.

- `warm_start`: Cuando se establece en `True`, reutiliza la solución de la llamada anterior para ajustar y agregar más estimadores al conjunto, de manera incremental. En este caso, se establece en `False`.

En resumen, este diccionario especifica todos los hiperparámetros utilizados para configurar un modelo de regresión con Gradient Boosting. Puedes ajustar estos valores según tus necesidades y preferencias específicas.

El código que has proporcionado crea un pipeline utilizando la clase `Pipeline` de scikit-learn. Un pipeline es una secuencia de pasos de procesamiento de datos y modelado que se pueden ejecutar de manera secuencial.

Aquí hay una explicación de cada paso en el pipeline:

1. **Imputer (imputador):**
   ```python
   ('imputer', SimpleImputer(missing_values=np.nan, strategy='mean'))
   ```
   Este paso utiliza `SimpleImputer` para tratar los valores faltantes en los datos. En este caso, los valores faltantes (NaN) se están reemplazando por la media de la columna correspondiente.

2. **Scaler (escalador):**
   ```python
   ('scaler', StandardScaler())
   ```
   Este paso utiliza `StandardScaler` para estandarizar (escalar) las características. La estandarización implica restar la media y dividir por la desviación estándar, lo que ayuda a que todas las características tengan la misma escala.

3. **Core Model (modelo principal):**
   ```python
   ('core_model', GradientBoostingRegressor(**original_params))
   ```
   Este es el paso final del pipeline y representa el modelo principal. En este caso, el modelo principal es un regresor de Gradient Boosting (`GradientBoostingRegressor`). Se configura con los parámetros especificados en el diccionario `original_params` que definiste anteriormente.

El pipeline completo se crea al pasar una lista de tuplas al constructor de `Pipeline`. Cada tupla contiene un nombre descriptivo para el paso y el objeto que realiza esa operación. Los pasos se ejecutan en orden, y el resultado de cada paso se pasa como entrada al siguiente.

Este tipo de pipelines es especialmente útil para organizar y estructurar el código cuando estás trabajando con modelos de machine learning, ya que permite encapsular todas las operaciones necesarias en un flujo de trabajo coherente. Además, facilita la reutilización y la reproducción del flujo de trabajo en nuevos conjuntos de datos.

Cross-validation con parametros originales

In [29]:
results_original_params = cross_validate(estimator, X, y, return_train_score=True, cv=10)  #La validación cruzada es una técnica comúnmente utilizada para evaluar el rendimiento de un modelo y verificar su capacidad para generalizar a datos no vistos. cv=10: Especifica la estrategia de validación cruzada, en este caso, se utiliza validación cruzada con 10 divisiones (10-fold cross-validation).

train_score_original_params = np.mean(results_original_params['train_score'])

test_score_original_params = np.mean(results_original_params['test_score'])

Ajuste de hiperparámetros

In [34]:
param_tuning = {
    'core_model__n_estimators': range(20, 101, 20),
    'core_model__learning_rate': [0.01, 0.1, 0.5]
}

grid_search = GridSearchCV(estimator, param_grid=param_tuning, scoring='r2', cv=5)
grid_search.fit(X_train, y_train)


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('imputer', SimpleImputer()),
                                       ('scaler', StandardScaler()),
                                       ('core_model',
                                        GradientBoostingRegressor(n_estimators=220))]),
             param_grid={'core_model__learning_rate': [0.01, 0.1, 0.5],
                         'core_model__n_estimators': range(20, 101, 20)},
             scoring='r2')

Resultados con mejores parametros

In [35]:
final_results = cross_validate(grid_search.best_estimator_,X_train,y_train, return_train_score=True, cv=7)

train_score_best_params = np.mean(final_results['train_score'])

test_score_best_params = np.mean(final_results['test_score'])

Impresion de resultados

In [36]:
print(f'Original train score: {train_score_original_params}')
print(f'Original test score: {test_score_original_params}')
print(f'Best train score: {train_score_best_params}')
print(f'Best test score: {test_score_best_params}')

Original train score: 0.9477358734252774
Original test score: 0.7517272536604832
Best train score: 0.9746637325085372
Best test score: 0.7424869359435994


Mejores parametros encontrados

In [37]:
best_params = grid_search.best_estimator_.get_params()
print('Best parameters: ')

print(best_params)

Best parameters: 
{'memory': None, 'steps': [('imputer', SimpleImputer()), ('scaler', StandardScaler()), ('core_model', GradientBoostingRegressor(learning_rate=0.5, n_estimators=60))], 'verbose': False, 'imputer': SimpleImputer(), 'scaler': StandardScaler(), 'core_model': GradientBoostingRegressor(learning_rate=0.5, n_estimators=60), 'imputer__add_indicator': False, 'imputer__copy': True, 'imputer__fill_value': None, 'imputer__keep_empty_features': False, 'imputer__missing_values': nan, 'imputer__strategy': 'mean', 'scaler__copy': True, 'scaler__with_mean': True, 'scaler__with_std': True, 'core_model__alpha': 0.9, 'core_model__ccp_alpha': 0.0, 'core_model__criterion': 'friedman_mse', 'core_model__init': None, 'core_model__learning_rate': 0.5, 'core_model__loss': 'squared_error', 'core_model__max_depth': 3, 'core_model__max_features': None, 'core_model__max_leaf_nodes': None, 'core_model__min_impurity_decrease': 0.0, 'core_model__min_samples_leaf': 1, 'core_model__min_samples_split': 2,

In [38]:
estimator.fit(X_train,y_train)

Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', StandardScaler()),
                ('core_model', GradientBoostingRegressor(n_estimators=220))])

In [39]:
estimator.score(X_test, y_test)

0.7398316408938105

Saving Model

In [40]:
from joblib import dump

In [45]:
ruta_modelo = r'C:/Users/matia/OneDrive/Escritorio/DataScience/Platzi/ML con python/Despliegue de modelos de ML/model/model.pkl'
dump(estimator, ruta_modelo)

['C:/Users/matia/OneDrive/Escritorio/DataScience/Platzi/ML con python/Despliegue de modelos de ML/model/model.pkl']